# Google Calendar to Gridy

- 指定したカレンダーのカレンダーIDを取得
- APIを使って情報を抜き出す
- Gridyにインポートできるように整形
- seleniumとかでぶち込む？（APIあるかな）

In [12]:
import json

from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import sys

In [5]:
# 抜き出すCalendarIDはJsonファイルで保存
with open("../calendar_info.json","r") as f:
    calendar_info = json.load(f)

```Python
calendar_info = {任意の名前: カレンダーID, ...}
```

In [11]:
# この辺りはテンプレ

# これらのスコープを変更する場合は、token.pickleというファイルを削除してください。
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

creds = None
# token.pickleファイルには、ユーザーのアクセストークンと
# リフレッシュトークンが格納されており、認証フローが初めて完了したときに
# 自動的に作成されます。
if os.path.exists('../token.pickle'):
    # gitにupする関係上ログイン関係ファイルは1階層下に配置している
    with open('../token.pickle', 'rb') as token:
        creds = pickle.load(token)
        
# 利用可能な（有効な）認証情報がない場合は、ユーザーのログインを許可します。
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '../credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('../token.pickle', 'wb') as token:
        pickle.dump(creds, token)
        
# たぶん、カレンダーだよって主張している
service = build('calendar', 'v3', credentials=creds)

In [20]:
# Calendar APIの呼び出し
# 現在時刻
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time

yotei = {}
# カレンダーIDすべての予定を抽出
for ID in calendar_info.values():
    events_result = service.events().list(calendarId=ID, timeMin=now,
                                        maxResults=100, singleEvents=True,
                                        orderBy='startTime').execute()
    for event in events_result.get('items', []):
        yotei[event["id"]] = {'summary':event['summary'],
                              "start":event["start"],
                              "end":event["end"]}

In [1]:
# yotei